# Module Import
## Module


* numpy : ndarray
* cv2 : image resize, 불러오기 등의 처리
* glob : 디렉토리 파일 탐색
* tensorflow : 모듈 설계
* itertools : list iterate

In [1]:
import numpy as np
import cv2
import sys
import os
import glob
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from itertools import chain, repeat, cycle
from sklearn.model_selection import train_test_split

# Colab 드라이브 마운트
##### 이후 $ cd gdrive/MyDrive/경로.. 로 기본 디렉토리 수정하여 사용
##### 이미지 데이터를 가지고 와서 train data 라벨링 진행해야 함

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
cd gdrive/MyDrive/KT

/content/gdrive/MyDrive/KT


# 이미지 불러오기
##### glob.glob(path) 함수 결과를 sort하여 img데이터 들어오는 순서 고정
##### cv2.imread() 함수로 img를 읽어오게 됨
  - cv2.IMREAD_COLOR : 컬러 이미지 불러올 때 쓰는 옵션
##### cv2.resize() : 크기 조절
  - dsize를 고정함으로써 원하는 크기로 불러온 이미지 데이터 수정 가능

##### 256.0으로 나누어 float데이터 타입을 유지하면서 normalize





In [4]:
# get img list from path
image_folders = sorted(glob.glob("Image/*"))

In [5]:
image_folders.remove('Image/info')

In [8]:
image_folders

['Image/01', 'Image/02', 'Image/03', 'Image/04']

In [41]:
files = []
each_class_num = []
for folder in image_folders:
  temp = sorted(glob.glob(folder + "/*.jpg"))
  temp.extend(sorted(glob.glob(folder + "/*.png")))
  temp.extend(sorted(glob.glob(folder + "/*.jpeg")))
  temp.extend(sorted(glob.glob(folder + "/*.jfif")))
  files.extend(temp)
  each_class_num.append(len(temp))

In [42]:
each_class_num

[163, 196, 196, 181]

In [43]:
# read image from files list & reshape
# train_img shape : ( , 256, 256, 3)
train_img = np.array(np.array([cv2.resize(cv2.imread(file, cv2.IMREAD_COLOR), dsize=(256, 256),
                                 interpolation=cv2.INTER_LINEAR).astype(np.float64) for file in files]))

In [44]:
image_num = train_img.shape[0]
labels = 4
train_label = np.array(list(chain.from_iterable((repeat(n, k) for (n, k) in zip(range(labels), each_class_num)))))

In [45]:
print(train_img.shape)
print(train_img[0].shape)
print(train_label.shape)

(736, 256, 256, 3)
(256, 256, 3)
(736,)


In [46]:
# normalize
train_img = train_img / (256.0)

In [47]:
x_train, x_test, y_train, y_test = train_test_split(train_img, train_label, test_size=0.1, stratify=train_label, random_state=34)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, stratify=y_train, random_state=12)

In [48]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_val.shape)
print(y_val.shape)

(595, 256, 256, 3)
(595,)
(74, 256, 256, 3)
(74,)
(67, 256, 256, 3)
(67,)


# Model Architecture

**코드가 조금 더러움 주의**

기본적으로 ResNet의 구조를 가지고 있는 NN으로 구성
## Hyperparameters
##### **수정 불가능한 hyperparameter**
  - input_img_shape : 256*256으로 고정
  - input_channel : RGB 3채널 이미지

##### **수정 가능한 hyperparmeter**
  - layer1_output_channel : layer1 의 output channel개수
  - layer1_kernel_size : ResNet 처음 conv kernel_size는 7*7 이용
  - layer2_output_channel : layer2 의 output channel개수 (== layer1_output_channel)
  - layer3_output_channel : layer3 의 output channel개수 (layer2_output_channel * 2)
  - res_kernel_size : resNet에서는 기본적으로 첫번째 conv 제외 3*3 kernel로 고정

## Layers
1. layer1 : convolution + pooling
2. layer2 : conv + norm + relu + conv + norm + relu + add
3. layer3 : conv + norm + relu + conv + norm + relu + identity + add
4. flatten : 분류기에 넣기 위해 flatten진행
5. classifier : 분류기, FC layer이용, ResNet에서는 FC layer의 Weight개수가 많으므로 한개의 FC layer만 사용

### 1*1 Convolution
layer2에서 사용하는 res_unit2의 경우, 이미지 데이터가 들어갈 때 layer2_output_channel개로 들어가지만, 나올 때 layer3_output_channel로 나오게 되는데, channel개수를 맞추어주기 위해서 convolution진행.
이미지 크기도 res_unit2의 첫번째 convolution에서 stride (2, 2)로 진행하므로 반으로 줄어들게 되므로 1*1 convolution도 stride (2, 2)로 진행

In [59]:
# Hyperparameters
global input_img_shape, input_channel, layer1_output_channel, layer1_kernel_size, res_kernel_size, layer2_output_channel, layer3_output_channel
input_img_shape = (256, 256, 3)
input_channel = 3
layer1_output_channel = 6
layer1_kernel_size = (7, 7)
layer2_output_channel = 6
layer3_output_channel = 12

# overall residual units num = num_residual_units * num_residual_layers 
res_kernel_size = (11, 11)

In [60]:
from tensorflow.python.keras.activations import softmax
class cnn_model(tf.keras.Model):
  global input_img_shape, input_channel, layer1_output_channel, layer1_kernel_size, res_kernel_size, layer2_output_channel, layer3_output_channel
  def __init__(self):
    super(cnn_model, self).__init__()
    
    self.conv1 = layers.Conv2D(layer1_output_channel, layer1_kernel_size, strides=(2, 2), 
                       padding='same', input_shape=input_img_shape, activation='relu')
    self.pool1 = layers.MaxPool2D(pool_size=(4, 4))
    self.conv2_t = layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same')
    self.batchnorm1 = layers.BatchNormalization()
    self.relu1 = layers.ReLU()
    self.conv2 = layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same')
    self.batchnorm2 = layers.BatchNormalization()
    self.relu2 = layers.ReLU()
    self.add1 = layers.Add()
    
    self.conv3 = layers.Conv2D(layer3_output_channel, res_kernel_size, strides=(2, 2), padding='same')
    self.batchnorm3 = layers.BatchNormalization()
    self.relu3 = layers.ReLU()
    
    self.conv4 = layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same')
    self.batchnorm4 = layers.BatchNormalization()
    self.relu4 = layers.ReLU()
    self.identity = layers.Conv2D(layer3_output_channel, (1, 1), padding='same', strides=(2, 2))
    self.add2 = layers.Add()
    
    self.flat = layers.Flatten()
    self.fc1 = layers.Dense(4)

  def call(self, x):
    x = self.conv1(x)
    x = self.pool1(x)
    x_t = self.conv2_t(x)
    x_t = self.batchnorm1(x_t)
    x_t = self.relu1(x_t)
    x_t = self.conv2(x_t)
    x_t = self.batchnorm2(x_t)
    x_t = self.relu2(x_t)
    x = self.add1([x, x_t])

    x_t = self.conv3(x)
    x_t = self.batchnorm3(x_t)
    x_t = self.relu3(x_t)
    x_t = self.conv4(x_t)
    x_t = self.batchnorm4(x_t)
    x_t = self.relu4(x_t)
    x = self.identity(x)
    x = self.add2([x, x_t])
    x = self.flat(x)
    x = self.fc1(x)
    probs = softmax(x)
    return probs

In [61]:
model = cnn_model()

model optimizer, loss등 정의 compile

In [62]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train

In [63]:
# Train Hyperparameters
epoch = 50
batch = 10

In [64]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_val.shape)
print(y_val.shape)

(595, 256, 256, 3)
(595,)
(74, 256, 256, 3)
(74,)
(67, 256, 256, 3)
(67,)


In [65]:
filename = 'checkpoint.ckpt'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=20)
# , callbacks=[checkpoint, earlystopping]

In [66]:
history = model.fit(x_train, y_train, epochs=epoch, validation_data=(x_val, y_val) , callbacks=[checkpoint, earlystopping])

Epoch 1/50
19/19 [==============================] - ETA: 0s - loss: 1.3399 - accuracy: 0.5109
Epoch 00001: val_loss improved from inf to 1.80360, saving model to checkpoint.ckpt
19/19 [==============================] - 3s 75ms/step - loss: 1.3399 - accuracy: 0.5109 - val_loss: 1.8036 - val_accuracy: 0.2537
Epoch 2/50
19/19 [==============================] - ETA: 0s - loss: 0.6203 - accuracy: 0.7815
Epoch 00002: val_loss improved from 1.80360 to 1.35779, saving model to checkpoint.ckpt
19/19 [==============================] - 1s 44ms/step - loss: 0.6203 - accuracy: 0.7815 - val_loss: 1.3578 - val_accuracy: 0.4179
Epoch 3/50
19/19 [==============================] - ETA: 0s - loss: 0.4989 - accuracy: 0.8185
Epoch 00003: val_loss improved from 1.35779 to 1.13716, saving model to checkpoint.ckpt
19/19 [==============================] - 1s 46ms/step - loss: 0.4989 - accuracy: 0.8185 - val_loss: 1.1372 - val_accuracy: 0.4925
Epoch 4/50
19/19 [==============================] - ETA: 0s - loss: 

In [67]:
model_fi = cnn_model()
model_fi.load_weights(filename)
model_fi.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [68]:
results = model_fi.evaluate(x_test, y_test)

3/3 [==============================] - 1s 60ms/step - loss: 0.3195 - accuracy: 0.9189


In [69]:
print('test loss, test acc:', results)

test loss, test acc: [0.31953859329223633, 0.9189189076423645]


In [70]:
model_fi.summary()

Model: "cnn_model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          multiple                  888       
                                                                 
 max_pooling2d_7 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 conv2d_43 (Conv2D)          multiple                  4362      
                                                                 
 batch_normalization_28 (Bat  multiple                 24        
 chNormalization)                                                
                                                                 
 re_lu_28 (ReLU)             multiple                  0         
                                                                 
 conv2d_44 (Conv2D)          multiple                  

In [ ]:
model_fi.save('cnnmodel')

In [ ]:
loaded = tf.keras.models.load_model('cnnmodel')

In [ ]:
loaded.evaluate(x_test, y_test)

1/1 [==============================] - 0s 430ms/step - loss: 0.4683 - accuracy: 0.9048


[0.4682854115962982, 0.9047619104385376]

#Dump

In [ ]:
model(train_img)

In [ ]:
# def residual_unit(self, x, in_channel, out_channel, kernel_size):
  #   if in_channel == out_channel:
  #     res1 = layers.Conv2D(out_channel, kernel_size, padding='same', strides=(1, 1))
  #   else:
  #     res1 = layers.Conv2D(out_channel, kernel_size, padding='same', strides=(2, 2))
  #   batch1 = layers.BatchNormalization()
  #   relu1 = layers.ReLU()
  #   res2 = layers.Conv2D(out_channel, kernel_size, padding='same')
  #   batch2 = layers.BatchNormalization()
  #   relu2 = layers.ReLU()

  #   x_t = res1(x)
  #   x_t = batch1(x_t)
  #   x_t = relu1(x_t)
  #   x_t = res2(x_t)
  #   x_t = batch2(x_t)
  #   x_t = relu2(x_t)

  #   if in_channel != out_channel:
  #     identity = layers.Conv2D(out_channel, (1, 1), padding='same', strides=(2, 2))
  #     x = identity(x)
  #   print(x_t.shape)
  #   print(x.shape)
  #   return x_t + x

  # channels = x.shape[3]
    # for l in range(self.num_residual_layers):
    #   for u in range(self.num_residual_units):
    #     x = self.residual_unit(x, channels, (lambda x,y:channels*2 if x != 0 and y == 0 else channels)(l, u), self.res_kernel_size)
    #     channels = channels * 2 if l != 0 and u == 0 else channels

In [ ]:
# model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape = (256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# model information
model.summary()

In [ ]:
#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#print(model(train_img))

In [ ]:
# in_channel : input image channel size, out_channel : output image channel of each unit
# kernel_size  ex) (3, 3), (5, 5), ..
class ResidualUnit(tf.keras.Model):
  def __init__(self, in_channel, out_channel, kernel_size, stride=False):
    super(ResidualUnit, self).__init__()
    if stride:
      self.conv1 = layers.Conv2D(out_channel, kernel_size, padding='same', activation='relu', stride=(2, 2))
    else:
      self.conv1 = layers.Conv2D(out_channel, kernel_size, padding='same', activation='relu')
    self.conv2 = layers.Conv2D(out_channel, kernel_size, padding='same', activation='relu')

    if in_channel ==  out_channel:
      self.identity = lambda x: x
    else:
      self.identity = layers.Conv2D(out_channel, (1, 1), padding='same')

  def call(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    return x + self.identity(x)

In [ ]:
class ResidualLayer(tf.keras.Model):
  def __init__(self, in_channel, out_channel, kernel_size, stride=False):
    super(ResidualLayer, self).__init__()
    # residualunit + unit + unit
    self.res1 = ResidualUnit(in_channel, out_channel, kernel_size, stride)
    self.res2 = ResidualUnit(out_channel, out_channel, kernel_size)
    self.res3 = ResidualUnit(out_channel, out_channel, kernel_size)
  
  def call(self, x):
    x = self.res1(x)
    x = self.res2(x)
    x = self.res3(x)
    return x

In [ ]:
from tensorflow.python.keras.activations import softmax
model = models.Sequential()
model.add(layers.Conv2D(layer1_output_channel, (11, 11), activation='relu', input_shape = (256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(layer2_output_channel, (7, 7), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(layer3_output_channel, (7, 7), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
from tensorflow.python.keras.activations import softmax
class cnn_model(tf.keras.Model):
  global input_img_shape, input_channel, layer1_output_channel, layer1_kernel_size, res_kernel_size, layer2_output_channel, layer3_output_channel
  def __init__(self):
    super(cnn_model, self).__init__()
    self.num_residual_units = 3 # number of units per layer
    self.num_residual_layers = 2 # number of layer per model
    self.res_kernel_size = (3, 3)

    self.layer1 = models.Sequential(
        [
         layers.Conv2D(layer1_output_channel, layer1_kernel_size, strides=(2, 2), 
                       padding='same', input_shape=input_img_shape, activation='relu'),
         layers.MaxPool2D(pool_size=(4, 4))]
    )

    self.res_unit1 = models.Sequential(
        [layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )
    '''

    self.res_unit2 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, strides=(2, 2), padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.res_unit3 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )'''

    #self.identity = layers.Conv2D(layer3_output_channel, (1, 1), padding='same', strides=(2, 2))

    self.flatten_layer = layers.Flatten()

    self.classifier = models.Sequential(
        [layers.Dense(4)]
    )

  def call(self, x):
    x = self.layer1(x)
    x_t = self.res_unit1(x)
    x = x + x_t
    #x_t = self.res_unit1(x)
    #x = x + x_t
    #x_t = self.res_unit1(x)
    #x = x + x_t
    #x_t = self.res_unit2(x)
    #x = self.identity(x) + x_t
    #x_t = self.res_unit3(x)
    #x = x + x_t
    #x_t = self.res_unit3(x)
    #x = x + x_t
    #x_t = self.res_unit3(x)
    #x = x + x_t
    x = self.flatten_layer(x)
    x = self.classifier(x)
    probs = softmax(x)
    return probs

In [ ]:
class cnn_model(tf.keras.Model):
  global input_img_shape, input_channel, layer1_output_channel, layer1_kernel_size, res_kernel_size, layer2_output_channel, layer3_output_channel
  def __init__(self):
    super(cnn_model, self).__init__()

    self.layer1 = models.Sequential(
        [layers.Conv2D(layer1_output_channel, layer1_kernel_size, strides=(2, 2), 
                       padding='same', input_shape=input_img_shape, activation='relu'),
         layers.MaxPool2D(pool_size=(2, 2))]
    )

    self.unit1 = models.Sequential(
        [layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )
    '''
    self.unit2 = models.Sequential(
        [layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.unit3 = models.Sequential(
        [layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )
    '''

    self.unit4 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, strides=(2, 2), padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    '''self.unit5 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.unit6 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )

    self.unit7 = models.Sequential(
        [layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU(),
         layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same'),
         layers.BatchNormalization(),
         layers.ReLU()
         ]
    )   '''
    self.identity = layers.Conv2D(layer3_output_channel, (1, 1), padding='same', strides=(2, 2))
    self.flatten_layer = layers.Flatten()

    self.classifier = models.Sequential(
        [layers.Dense(4)]
    )

    

  def call(self, x):
    x = self.layer1(x)
    x = self.unit1(x)
    #x = x + x_t
    #x_t = self.unit2(x)
    #x = x + x_t
    #x_t = self.unit3(x)
    #x = x + x_t
    x = self.unit4(x)
    #x = self.identity(x) + x_t
    #x_t = self.unit5(x)
    #x = x + x_t
    #x_t = self.unit6(x)
    #x = x + x_t
    #x_t = self.unit7(x)
    #x = x + x_t
    x = self.flatten_layer(x)
    x = self.classifier(x)
    probs = tf.nn.softmax(x)
    return probs

In [ ]:
from tensorflow.python.keras.activations import softmax
class cnn_model(tf.keras.Model):
  global input_img_shape, input_channel, layer1_output_channel, layer1_kernel_size, res_kernel_size, layer2_output_channel, layer3_output_channel
  def __init__(self):
    super(cnn_model, self).__init__()
    
    self.conv1 = layers.Conv2D(layer1_output_channel, layer1_kernel_size, strides=(2, 2), 
                       padding='same', input_shape=input_img_shape, activation='relu')
    self.pool1 = layers.MaxPool2D(pool_size=(4, 4))
    self.conv2_t = layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same')
    self.batchnorm1 = layers.BatchNormalization()
    self.relu1 = layers.ReLU()
    self.conv2 = layers.Conv2D(layer2_output_channel, res_kernel_size, padding='same')
    self.batchnorm2 = layers.BatchNormalization()
    self.relu2 = layers.ReLU()
    self.add1 = layers.Add()
    
    self.conv3 = layers.Conv2D(layer3_output_channel, res_kernel_size, strides=(2, 2), padding='same')
    self.batchnorm3 = layers.BatchNormalization()
    self.relu3 = layers.ReLU()
    
    self.conv4 = layers.Conv2D(layer3_output_channel, res_kernel_size, padding='same')
    self.batchnorm4 = layers.BatchNormalization()
    self.relu4 = layers.ReLU()
    self.identity = layers.Conv2D(layer3_output_channel, (1, 1), padding='same', strides=(2, 2))
    self.add2 = layers.Add()
    
    self.flat = layers.Flatten()
    self.fc1 = layers.Dense(4)

  def call(self, x):
    x = self.conv1(x)
    x = self.pool1(x)
    x_t = self.conv2_t(x)
    x_t = self.batchnorm1(x_t)
    x_t = self.relu1(x_t)
    x_t = self.conv2(x_t)
    x_t = self.batchnorm2(x_t)
    x_t = self.relu2(x_t)
    x = self.add1([x, x_t])

    x_t = self.conv3(x_t)
    x_t = self.batchnorm3(x_t)
    x_t = self.relu3(x_t)
    x_t = self.conv4(x_t)
    x_t = self.batchnorm4(x_t)
    x_t = self.relu4(x_t)
    x = self.identity(x)
    x = self.add2([x, x_t])
    x = self.flat(x)
    x = self.fc1(x)
    probs = softmax(x)
    return probs